In [1]:
from pulp import LpProblem, LpVariable, lpSum, LpMinimize
import pandas as pd

In [2]:
df_vehiculos = pd.read_excel('../../Datos_P1/df_vehicle.xlsx')
df_clientes = pd.read_excel('../../Datos_P1/df_orders.xlsx')
df_distancias = pd.read_excel('../../Datos_P1/df_distance_km.xlsx')

df_distancias.index = df_distancias.columns

In [3]:
# Preparación datos para PuLP

capacidades = dict(zip(df_vehiculos["vehiculo_id"], df_vehiculos["capacidad_kg"]))
costos = dict(zip(df_vehiculos["vehiculo_id"], df_vehiculos["costo_km"]))
autonomias = dict(zip(df_vehiculos["vehiculo_id"], df_vehiculos["autonomia_km"]))

pedidos = dict(zip(df_clientes["cliente"], df_clientes["order_demand"]))
#pedidos["Almacén"] = 0

distancias = df_vehiculos.stack().to_dict()  # Convierte a un diccionario (i, j): distancia

clientes = df_clientes["cliente"].tolist() #+ ["Almacén"]
vehiculos = df_vehiculos["vehiculo_id"].tolist()

In [4]:
# Crear el problema
problema = LpProblem("Ruteo_de_Vehiculos", LpMinimize)


# Variables de decisión
x = LpVariable.dicts("x", [(i, j, k) for i in clientes for j in clientes for k in vehiculos], cat="Binary")
q = LpVariable.dicts("q", [(i, k) for i in clientes for k in vehiculos], lowBound=0)

# Función objetivo: minimizar costo total
problema += lpSum(
    x[i, j, k] * costos[k] * distancias.get((i, j), 0)
    for i in clientes for j in clientes for k in vehiculos
)

# Restricciones
# Cada cliente debe ser atendido una vez
for i in clientes:
    problema += lpSum(x[i, j, k] for j in clientes for k in vehiculos) == 1

# Respetar la capacidad de los vehículos
for k in vehiculos:
    problema += lpSum(q[i, k] for i in clientes) <= capacidades[k]

# Satisfacer la demanda de los clientes
for i in clientes:
    problema += lpSum(q[i, k] for k in vehiculos) == pedidos[i]

# Autonomía de los vehículos
for k in vehiculos:
    problema += lpSum(x[i, j, k] * distancias.get((i, j), 0) for i in clientes for j in clientes) <= autonomias[k]

# Flujo de vehículos: entrada = salida
for i in clientes:
    for k in vehiculos:
        problema += lpSum(x[i, j, k] for j in clientes) == lpSum(x[j, i, k] for j in clientes)

# Resolver el problema
problema.solve()

# Resultados
for k in vehiculos:
    print(f"Vehículo {k}:")
    for i in clientes:
        for j in clientes:
            if x[i, j, k].value() == 1:
                print(f"  {i}")

Vehículo 1:
  Cliente_1
  Cliente_4
  Cliente_15
  Cliente_16
Vehículo 2:
  Cliente_7
Vehículo 3:
  Cliente_2
  Cliente_12
  Cliente_18
Vehículo 4:
  Cliente_3
  Cliente_5
Vehículo 5:
  Cliente_6
  Cliente_8
  Cliente_9
  Cliente_10
  Cliente_11
  Cliente_17
  Cliente_19
Vehículo 6:
  Cliente_13
  Cliente_14
  Cliente_20
